## Load Dataset

In [1]:
# Uncomment if running on Google Colab
# %cd drive/My Drive/Maritime_Anomaly

/content/drive/My Drive/Maritime_Anomaly


In [2]:
import pandas as pd
import numpy as np
import sys
import os

In [3]:
df = pd.read_csv("AIS_2017_01_Zone01.csv")

df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo
0,366940480,2017-01-04T11:39:36,52.48730,-174.02316,10.0,-140.7,267.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0
1,366940480,2017-01-04T11:40:45,52.48718,-174.02835,10.0,-141.6,266.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0
2,366940480,2017-01-04T11:42:26,52.48705,-174.03608,10.0,-142.3,267.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0
3,366940480,2017-01-04T13:51:07,52.41575,-174.60041,9.1,-154.0,251.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0
4,366940480,2017-01-04T13:55:17,52.41311,-174.61718,9.1,-157.3,251.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0


## Entity Attribute Explanations

https://api.vtexplorer.com/docs/response-ais.html

| Field |	Type |	Description|
--- | --- | ---
MMSI |	integer |	MMSI number of the vessel (AIS identifier)
TIMESTAMP |	datetime |	Date and Time (in UTC) when position was recorded by AIS
LATITUDE |	float |	Geographical latitude (WGS84)
LONGITUDE |	float |	Geographical longitude (WGS84)
COURSE |	float |	Course over ground (degrees)
SPEED |	float |	Speed over ground (knots)
HEADING |	integer |	Heading (degrees) of the vessel's hull. A value of 511 indicates there is no heading data.
NAVSTAT |	integer |	Navigation status according to AIS Specification
IMO |	integer |	IMO number of the vessel
NAME |	text |	Name of the vessel
CALLSIGN |	text |Callsign of the vessel
TYPE |	integer |	Type of the vessel according to AIS Specification

## Data Exploration

In [4]:
dates = df['BaseDateTime'].str[:10].unique()
vessels_mmsi = df['MMSI'].unique()

In [5]:
vessel_gps_list = []

for vessel in vessels_mmsi:
  d = df.loc[df['MMSI'] == vessel]
  d['BaseDateTime'] = pd.to_datetime(d['BaseDateTime'])
  d['LAT'] = d['LAT'].astype(float)
  d['LON'] = d['LON'].astype(float)
  d = d.sort_values(by=['BaseDateTime'])
  d = d.reset_index(drop=True)
  d['Index'] = d.index
  vessel_gps_list.append(d)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

## Construct Test Data

*   Take 1 subset (100 points) of MMSI "366940480" as sonar sub "SONAR"
*   Take 1 subset (100 points) of MMSI "366940480" as unknown vessel "VESSEL_1"
*   Both assumed to have known GPS locations, conversion to bearing representation below
*   Using same number of datapoints for each, ignoring time for now. Strict imposition of time could be done, unneccesary for now.

In [6]:
!pip install mapboxgl
api_key = 'insert_mapbox_key_here'

In [7]:
# creating subsets

index = 0

vessel_sonar = vessel_gps_list[index].loc[130:230].reset_index(drop=True)
vessel_sonar['NAME'] = 'SONAR'

vessel_1 = vessel_gps_list[index].loc[1270:1370].reset_index(drop=True)
vessel_1['NAME'] = 'VESSEL_1'

# combined set for visualisation
vessel_viz = pd.concat([vessel_sonar, vessel_1]).reset_index(drop=True)

In [8]:
vessel_viz

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,Index,NAME
0,366940480,2017-01-04 15:52:17,52.32476,-175.08521,9.4,-158.6,251.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,130,SONAR
1,366940480,2017-01-04 15:53:27,52.32379,-175.08993,9.4,-157.4,252.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,131,SONAR
2,366940480,2017-01-04 15:54:38,52.32280,-175.09469,9.6,-159.8,251.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,132,SONAR
3,366940480,2017-01-04 15:55:44,52.32190,-175.09929,9.6,-153.8,255.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,133,SONAR
4,366940480,2017-01-04 15:56:46,52.32108,-175.10356,9.7,-156.2,250.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,134,SONAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,366940480,2017-01-20 22:53:54,52.37225,-175.11987,10.1,79.2,74.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,1366,VESSEL_1
198,366940480,2017-01-20 22:55:33,52.37337,-175.11269,9.8,72.7,69.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,1367,VESSEL_1
199,366940480,2017-01-20 22:57:15,52.37447,-175.10522,10.1,77.4,71.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,1368,VESSEL_1
200,366940480,2017-01-20 22:58:24,52.37532,-175.10033,10.0,74.0,75.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,1369,VESSEL_1


In [9]:
from mapboxgl.utils import create_color_stops, df_to_geojson, color_map
from mapboxgl.viz import CircleViz
from mapboxgl.viz import *
from mapboxgl.utils import *

# map center
center_lon = vessel_viz['LON'].iloc[0]
center_lat = vessel_viz['LAT'].iloc[0]

# create geojson file output
vessel_tracks = df_to_geojson(
      vessel_viz,
      properties=['NAME', 'MMSI', 'BaseDateTime', 'SOG', 'COG', 'Heading', 'Status', 'Index'],
      lat='LAT',
      lon='LON',
      precision=5
)

# color mapping
category_color_stops = [['SONAR', '#FF5733'],  
                        ['VESSEL_1', '#399cf1']]

viz = CircleViz(
    vessel_tracks,
    access_token=api_key,
    radius=1,
    height='400px',
    color_property='NAME',
    # label_property='NAME',
    color_default='grey',
    color_function_type='match',
    color_stops=category_color_stops,
    center = (center_lon, center_lat),
    zoom = 10,
    below_layer = 'waterway-label'
)
viz.show()

# Data Transformation

## Bearing: Sonar Sub to Vessel 1 (1 Vessel Implementation)

In [10]:
import math
from numpy import arctan2, random, sin, cos, degrees
import geopy.distance

In [11]:
vessel_sonar['BEARING'] = 511
vessel_1['BEARING'] = 511

In [12]:
def get_bearing(lat1, long1, lat2, long2):
  # 1 = sonar, 2 = vessel
  dLon = (long2 - long1)
  x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
  y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
  brng = arctan2(x,y)
  brng = degrees(brng)
  return brng

In [13]:
for i, row in vessel_sonar.iterrows():
  lat_sonar = row['LAT']
  lon_sonar = row['LON']
  lat_vessel = vessel_1.iloc[i]['LAT']
  lon_vessel = vessel_1.iloc[i]['LON']
  brng = get_bearing(lat_sonar, lon_sonar, lat_vessel, lon_vessel)
  row['BEARING'] = brng
  vessel_sonar.loc[i] = row

vessel_1['BEARING'] = vessel_1['BEARING'].apply(lambda x: -x)

In [16]:
vessel_sonar.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,Index,NAME,BEARING
0,366940480,2017-01-04 15:52:17,52.32476,-175.08521,9.4,-158.6,251.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,130,SONAR,-92.645366
1,366940480,2017-01-04 15:53:27,52.32379,-175.08993,9.4,-157.4,252.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,131,SONAR,-92.425018
2,366940480,2017-01-04 15:54:38,52.32280,-175.09469,9.6,-159.8,251.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,132,SONAR,-92.217072
3,366940480,2017-01-04 15:55:44,52.32190,-175.09929,9.6,-153.8,255.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,133,SONAR,-91.998663
4,366940480,2017-01-04 15:56:46,52.32108,-175.10356,9.7,-156.2,250.0,EARLY DAWN,IMO7821130,WDB7319,1001.0,undefined,32.95,8.82,4.0,31.0,134,SONAR,-91.780248
